In [ ]:
#!pip install rasterio
#!pip install geopandas

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from shapely.geometry import Polygon
import cv2
from clasificacion_chinchetas import *
import tqdm
import numpy as np
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
import sys
from shapely.geometry import mapping
from shapely import geometry
import cv2
import numpy as np



In [ ]:
def matchingTemplate(img_rgb,template,threshold = .7):
    '''
(Function)
(Parameters)
    - threshold: Porcentaje de coincidencia 
    '''
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    w, h = template.shape[::-1]
    res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
    loc = np.where( res >=threshold)
    startpoints=[]
    endpoints=[]
    thresh=[]
    x=-10
    y=-10
    for i,pt in enumerate(zip(*loc[::-1])):
        #if i==0:
        x=pt[0]
        y=pt[1]
        cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
        cv2.putText(img_rgb,str(i) ,(x,y) , cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
        startpoints.append(pt)
        endpoints.append((pt[0] + w, pt[1] + h))
        val=np.array(loc)[:,i]
        thresh.append(res[val[0],val[1]])
        if (pt[0]!=x and pt[0]+1!=x and pt[0]-1!=x and pt[0]+2!=x and pt[0]-2!=x) and (pt[1]!=y and pt[1]+1!=y and pt[1]-1!=y and pt[1]+2!=y and pt[1]-2!=y):
            x=pt[0]
            y=pt[1]
            cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
            cv2.putText(img_rgb,str(i) ,(x,y) , cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
            startpoints.append(pt)
            endpoints.append((pt[0] + w, pt[1] + h))
            val=np.array(loc)[:,i]
            thresh.append(res[val[0],val[1]])
    return img_rgb,startpoints,endpoints,thresh

def Parametro_raster(raster,metros=120):
    """Se obtienen las dimensiones del raster y del cuadro de corte"""
    gdal_interpeter = gdal.Open(raster)
    width = gdal_interpeter.RasterXSize
    height = gdal_interpeter.RasterYSize
    coordenadas_gdal = gdal_interpeter.GetGeoTransform()
    minx = coordenadas_gdal[0]
    miny = coordenadas_gdal[3] + width*coordenadas_gdal[4] + height*coordenadas_gdal[5] 
    maxx = coordenadas_gdal[0] + width*coordenadas_gdal[1] + height*coordenadas_gdal[2]
    maxy = coordenadas_gdal[3]
    src_raster_path = raster
    src=rasterio.open(src_raster_path)
    H,W=src.shape
    dim=int(np.ceil(map_d(minx+metros,minx,maxx,0,W)))
    alto=np.max([1,int(np.floor(H/dim))])
    ancho=np.max([1,int(np.floor(W/dim))])
    return alto,ancho,dim,src.crs,H,W,minx,maxx,miny,maxy

def map_d(x, in_min, in_max, out_min, out_max):
    """Genera una interpolacion para pasar de un rango a otro"""
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

def shape_transform(shape):
    """Convierte el shape en dataframe de coordenadas que engloba el polygon del shape para delimitar el raster"""
    c=[]
    angulo_manzana=[]

    # for manzana in range(len(shape)):
    #     proyecciones1=mapping(shape['geometry'][manzana]).get('coordinates')
    #     angulos=[]
    #     d=[]
    #     poly=pd.DataFrame(proyecciones1[0])
    #     for point in range(1,len(poly)):
    #         d.append(((poly[1][point]-poly[1][point-1])**2+(poly[0][point]-poly[0][point-1])**2))
    #         angulos.append(math.atan(((poly[1][point]-poly[1][point-1])/(poly[0][point]-poly[0][point-1])))*180/math.pi)
    #     angulo_manzana.append(angulos[d.index(max(d))])
    shape["angulo_manzana"]=0
    shape["geometry"]=shape["geometry"].envelope
    for manzana in range(len(shape)):
        proyecciones1=mapping(shape['geometry'][manzana]).get('coordinates')
        proyecciones=proyecciones1[0]
        point1=np.min((proyecciones,proyecciones),axis=1)[0]
        min_y,min_x=point1[0],point1[1]
        point2=np.max((proyecciones,proyecciones),axis=1)[0]
        max_y,max_x=point2[0],point2[1]
        c.append(','.join([str(min_y),str(min_x),str(max_y),str(max_x)]))
    shape1=pd.DataFrame()
    shape1['points']=c
    shape1=shape1['points'].str.split(',',expand=True)
    shape1=shape1.astype({0:'float64',1:'float64',2:'float64',3:'float64'})
    shape1["cve_cat"]=shape["cve_cat"]
    shape1["angulo_manzana"]=shape["angulo_manzana"]
    shape=shape1
    return shape

def rotacion_detect(startpoint,endpoint,angle,proyecciones,w,h,dim):
    """Rotata el cuadro detectado en el sistema de coordenadas inicial"""
    point1=np.min((proyecciones,proyecciones),axis=1)[0]
    min_y,min_x=point1[0],point1[1]
    point2=np.max((proyecciones,proyecciones),axis=1)[0]
    max_y,max_x=point2[0],point2[1]
    min_y,min_x,max_y,max_x,proyecciones
    tipos=["casa","terreno"]
    y1,x1=startpoint
    y2,x2=endpoint
    x1,y1=(x1)*2-1,(y1)*2-1
    x2,y2=(x2)*2-1,(y2)*2-1
    angle=angle*math.pi/180
#     x1,y1=x1*(w/dim),y1*(h/dim)
#     x2,y2=x2*(w/dim),y2*(h/dim)
    #x_p, y_p son los puntos de un rectangulo en el orden inverso al manecillas del reloj
    x1p=max_x-((x1*math.cos(angle)-y1*math.sin(angle)+1)/2)*(max_x-min_x)
    y1p=min_y+((x1*math.sin(angle)+y1*math.cos(angle)+1)/2)*(max_y-min_y)
    x2p=max_x-((x2*math.cos(angle)-y1*math.sin(angle)+1)/2)*(max_x-min_x)
    y2p=min_y+((x2*math.sin(angle)+y1*math.cos(angle)+1)/2)*(max_y-min_y)
    x3p=max_x-((x2*math.cos(angle)-y2*math.sin(angle)+1)/2)*(max_x-min_x)
    y3p=min_y+((x2*math.sin(angle)+y2*math.cos(angle)+1)/2)*(max_y-min_y)
    x4p=max_x-((x1*math.cos(angle)-y2*math.sin(angle)+1)/2)*(max_x-min_x)
    y4p=min_y+((x1*math.sin(angle)+y2*math.cos(angle)+1)/2)*(max_y-min_y)
    return Polygon(((y1p,x1p),(y2p,x2p),(y3p,x3p),(y4p,x4p),(y1p,x1p)))

def cv2_imshow(image):
    cv2.namedWindow('name', cv2.WINDOW_NORMAL)
    cv2.imshow('name',image)
    cv2.waitKey()
    cv2.destroyAllWindows()